In [1]:
from pynq import Overlay
Overlay("base.bit").download()

#Modify your target temperature here if you wish
target_temperature = 23

#Do NOT modify the following codes.
frame_in_w = 640
frame_in_h = 480
from pynq.drivers.video import Frame
import cv2
import numpy as np
from pynq.iop import Pmod_ALS
from pynq.iop import PMODB
from pynq.iop import Pmod_TMP2
from pynq.iop import PMODA
from time import sleep
from pynq.board import LED
from pynq.board import Button
from pynq.board import Switch

systemOn = False
while(not systemOn):
    if (Switch(0).read()):
        systemOn = True
    print("Waiting to start.")
    sleep(0.5)

print("System initializing...")
videoIn = cv2.VideoCapture(0)
videoIn.set(cv2.CAP_PROP_FRAME_WIDTH, frame_in_w);
videoIn.set(cv2.CAP_PROP_FRAME_HEIGHT, frame_in_h);
ret, frame_vga = videoIn.read()



print("System is on.")
engineOn = False
headbeam = False
air_conditioning = False
air_volume_level = 0 #varies between 0 - 10
circulation_mode = 0 #inner = 1, outside = 0
circulation_time = 0 #chane mode tempearily if higher than 3000

while (systemOn):
    #Awaiting start.
    print("Capture device is open: " + str(videoIn.isOpened()))
    if (not videoIn.isOpened()):
        print("Trying to restart the device.")
        videoIn.release()
        ret, frame_vga = videoIn.read()
        videoIn = cv2.VideoCapture(0)
        videoIn.set(cv2.CAP_PROP_FRAME_WIDTH, frame_in_w);
        videoIn.set(cv2.CAP_PROP_FRAME_HEIGHT, frame_in_h);
        if (not videoIn.isOpened()):
            print("Failed to start the capture device. System is turned off.")
            systemOn = False
    print("Please step on break to start the engine(simulated by BTN3).")
    while (not engineOn and systemOn):
        foundFace = 0
        while(Button(3).read() and not engineOn):
            ret, frame_vga = videoIn.read()
            if (ret):
                np_frame = frame_vga
                face_cascade = cv2.CascadeClassifier(
                                        './Examples/data/haarcascade_frontalface_default.xml')
                eye_cascade = cv2.CascadeClassifier(
                                        './Examples/data/haarcascade_eye.xml')
                gray = cv2.cvtColor(np_frame, cv2.COLOR_BGR2GRAY)
                faces = face_cascade.detectMultiScale(gray, 1.3, 5)

                if not(len(faces)==0):
                    foundFace += 1

                print(foundFace)
                sleep(0.1)
                
                if (foundFace>=3):
                    #send signal to the car to turn on the engine. However, signals can vary between cars, so it has not been implemented. 
                    engineOn = True
                    print("Face detected. Engine should be on. Shift out from Park to stop face recognition. ")
                
                if (not Switch(0).read()):
                    systemOn = False
                    videoIn.release()
                    print("System exited manually.")
                    
        if (not Switch(0).read()):
            systemOn = False
            videoIn.release()
            print("System exited manually.")

    if (systemOn):
        #Engine started. Other functions are ready. 
        print("ALS and TMP sensors are now working.")

    #temperary counters    
    hbmStatusCounter = 0
    tmpStatusCounter = 0
    disableInner = False
    faceNotFound = 0
    while (engineOn and systemOn):       
        
        #Temperature Sensors
        #mytmp = Pmod_TMP2(PMODA)
        #temperature = mytmp.read()
        #print("Current inside temperature: ", temperature, "C")
        #tmpdiff = temperature - target_temperature
        #if (tmpdiff > 1.0):
            #tmpStatusCounter += 1
            #if (tmpStatusCounter>10):
                #start AC
                #ac = True
                #if (tmpdiff > 5):
                    #circulation_time += 1
                    #if (not disableInner & circulation_mode == 0):
                        #circulation_mode = 1
                #if (tmpdiff > 10): 
                    #tmpdiff = 10
                #air_volume_level = int(tmpdiff)
                #tmpStatusCounter = 0
                #print("Turn on AC. Air Volume Level: ",air_volume_level)
        #if (tmpdiff < -1.0):
            #tmpStatusCounter -= 1
            #if (tmpStatusCounter<-10):
                #turn off AC
                #ac = False
                #circulation_mode = 0
                #if (tmpdiff < -10):
                    #tmpdiff = -10
                #air_volume_level = int(abs(diff))
                #tmpStatusCounter = 0
                #print("Turn off AC. Air Volume Level: ",air_volume_level)
        #if (not disableInner && circulation_time > 3000):
            #disableInner = True
            #circulation_mode = 0
            #circulation_time = 0
        #elif (disableInner && circulation_time > 200):
            #disableInner = False
            #circulation_time = 0
        
        #Ambient Light Sensors
        my_als = Pmod_ALS(PMODB)
        lil = my_als.read()/10
        #print("Current outside luminous index level: ", lil)
        if (lil > 2.0 and headbeam):
            hbmStatusCounter+=1
            if (hbmStatusCounter>5):
                #turn off headbeam
                headbeam = False
                hbmStatusCounter = 5
                print("Turn off headbeam.")
        if (lil < 2.0 and not headbeam):
            hbmStatusCounter-=1
            if (hbmStatusCounter<-5):
                #turn on headbeam
                headbeam = True
                hbmStatusCounter = -5
                print("Turn on headbeam.")

        #Automatic Car Stop
        if (Switch(1).read()):
            ret, frame_vga = videoIn.read()
            if (ret):
                np_frame = frame_vga
                face_cascade = cv2.CascadeClassifier(
                                        './Examples/data/haarcascade_frontalface_default.xml')
                eye_cascade = cv2.CascadeClassifier(
                                        './Examples/data/haarcascade_eye.xml')
                gray = cv2.cvtColor(np_frame, cv2.COLOR_BGR2GRAY)
                faces = face_cascade.detectMultiScale(gray, 1.3, 5)

                if not(len(faces)==0):
                    faceNotFound = 0
                else:
                    faceNotFound += 1
            print("Have not seen faces for ",faceNotFound," frames.")
            if (faceNotFound>5):
                #send signal to the car to turn off the engine. However, signals can vary between cars, so it has not been implemented. 
                engineOn = False
                videoIn.release()
                print("Engine turned off automatically.")

        if (not Switch(0).read()):
            systemOn = False
            videoIn.release()
            print("System exited manually.")
            
        sleep(0.1)

Waiting to start.
System initializing...
System is on.
Capture device is open: True
Please step on break to start the engine(simulated by BTN3).
0
1
1
1
2
3
Face detected. Engine should be on. Shift out from Park to stop face recognition. 
ALS and TMP sensors are now working.
Have not seen faces for  0  frames.
Have not seen faces for  0  frames.
Have not seen faces for  0  frames.
Have not seen faces for  0  frames.
Have not seen faces for  0  frames.
Have not seen faces for  0  frames.
Have not seen faces for  0  frames.
Have not seen faces for  0  frames.
Have not seen faces for  1  frames.
Have not seen faces for  2  frames.
Have not seen faces for  3  frames.
Have not seen faces for  4  frames.
Have not seen faces for  5  frames.
Have not seen faces for  6  frames.
Engine turned off automatically.
Capture device is open: False
Trying to restart the device.
Please step on break to start the engine(simulated by BTN3).
1
2
3
Face detected. Engine should be on. Shift out from Park to 